Explore TwitterAPI
====

Based on this code: http://geduldig.github.io/TwitterAPI/faulttolerance.html

In [73]:
reset -fs

In [74]:
from itertools import islice
from time import sleep

from credentials import credentials
from ftfy import fix_text
from textblob import TextBlob
from TwitterAPI import TwitterAPI

In [75]:
creds = credentials.require(['access_token', 
                             'access_token_secret', 
                             'consumer_key',
                             'consumer_secret'])

api = TwitterAPI(creds.consumer_key,
                 creds.consumer_secret,
                 creds.access_token,
                 creds.access_token_secret)

In [76]:
def tweets(query):
    "A generator of tweets"
    while True:
        try:
            iterator = api.request('search/tweets', {'q':query}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield(item['text'])
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [50]:
# Grab some tweets

tweets_about_pizza = tweets(query='pizza')
list(islice(tweets_about_pizza, 5))

['Mejor me voy a comer pizza 😁',
 'Qué hago comiendo pizza a esta hora?😂😂🍕',
 'pizza pizza pizza via /r/Jokes https://t.co/feMKdiBqG1',
 "RT @MAGCONUpdates: What's better than a little rap break between races? Everyone loves PIZZA! #MagconTakeover #MagconSanFrancisco https://t…",
 'RT @auhstralian: Remember when Ashton was all sweet and tweeted about animals and then Luke tweeted about hot dogs. https://t.co/hmhSomUgLM']

In [77]:
# Stream tweets and tag the words as they go by
tweets_about_beer = tweets(query='beer')

for _ in range(2):
    blob = TextBlob(fix_text(next(tweets_about_beer)))
    print("Tweet: \t", blob.raw)
    print("POS Tags:",) # Part of Speech 
    [print("\t", token) for token in blob.tags]
    print('#'*40)
    sleep(1)

Tweet: 	 RT @SMM_sportradio: 'ฟีฟ่า'อนุมัติ!เปลี่ยนสำรองคนที่ 4 ช่วงต่อเวลาพิเศษ https://t.co/v60AFdJEI1 #FM96 #SMMSPORT
POS Tags:
	 ('RT', 'NNP')
	 ('@', 'NNP')
	 ('SMM_sportradio', 'NNP')
	 ("'ฟีฟ่า'อนุมัติ", 'NN')
	 ('4', 'CD')
	 ('ช่วงต่อเวลาพิเศษ', 'JJ')
	 ('https', 'NN')
	 ('//t.co/v60AFdJEI1', 'JJ')
	 ('FM96', 'NNP')
	 ('SMMSPORT', 'NNP')
########################################
Tweet: 	 RT @BirdlandBrewing: 10 Beers on Tap , Hoot Beer, 4 Gourmet Sandwiches, 6 Artisan Salads, Decadent Mac-n-Cheese and One of a Kind Desserts.…
POS Tags:
	 ('RT', 'NNP')
	 ('@', 'NNP')
	 ('BirdlandBrewing', 'NNP')
	 ('10', 'CD')
	 ('Beers', 'NNS')
	 ('on', 'IN')
	 ('Tap', 'NNP')
	 ('Hoot', 'NNP')
	 ('Beer', 'NNP')
	 ('4', 'CD')
	 ('Gourmet', 'NNP')
	 ('Sandwiches', 'NNP')
	 ('6', 'CD')
	 ('Artisan', 'NNP')
	 ('Salads', 'NNP')
	 ('Decadent', 'NNP')
	 ('Mac-n-Cheese', 'NNP')
	 ('and', 'CC')
	 ('One', 'CD')
	 ('of', 'IN')
	 ('a', 'DT')
	 ('Kind', 'NNP')
	 ('Desserts.…', 'NNP')
########################

---
__The internet also has Rainbow Butterfly Unicorn Kittens:__

![](https://s-media-cache-ak0.pinimg.com/736x/bf/3f/4c/bf3f4c4e4cbc909f957f939bb6bc7cc6.jpg)